# SRA Testing Module
For scratch testing of the SRA module.

In [1]:
# Enable autorealod so that modified files can be re-loaded
# with out restarting the kernel.
%load_ext autoreload
%autoreload 2

In [2]:
%pwd

'C:\\Users\\Tyler\\git\\pynome\\tests'

In [3]:
# Add the path of the pynome project.
import os
import sys
sys.path.insert(0, os.path.abspath('..'))

In [6]:
from pynome.sra import (
    build_sra_query_string,
    run_sra_query,
    fetch_sra_info,
)

In [7]:
search = build_sra_query_string('3702')

In [8]:
result = run_sra_query(search)

In [9]:
type(result)

collections.OrderedDict

In [10]:
result.keys()

odict_keys(['eSearchResult'])

In [11]:
result['eSearchResult'].keys()

odict_keys(['Count', 'RetMax', 'RetStart', 'IdList', 'TranslationSet', 'TranslationStack', 'QueryTranslation'])

In [13]:
# print(result['eSearchResult']['IdList']['Id'])

In [15]:
fetch_result = fetch_sra_info(result['eSearchResult']['IdList']['Id'][0])

In [17]:
fetch_result.keys()

odict_keys(['EXPERIMENT_PACKAGE_SET'])

In [32]:
fetch_result['EXPERIMENT_PACKAGE_SET']['EXPERIMENT_PACKAGE']

OrderedDict([('EXPERIMENT',
              OrderedDict([('@accession', 'SRX3422726'),
                           ('@alias', '9'),
                           ('IDENTIFIERS',
                            OrderedDict([('PRIMARY_ID', 'SRX3422726'),
                                         ('SUBMITTER_ID',
                                          OrderedDict([('@namespace',
                                                        'SUB3255603'),
                                                       ('#text', '9')]))])),
                           ('TITLE',
                            'Plant sample from Arabidopsis thaliana WTA-3'),
                           ('STUDY_REF',
                            OrderedDict([('@accession', 'SRP125650'),
                                         ('IDENTIFIERS',
                                          OrderedDict([('PRIMARY_ID',
                                                        'SRP125650'),
                                                       ('EX

In [20]:
fetch_result['EXPERIMENT_PACKAGE_SET']['EXPERIMENT_PACKAGE'].keys()

odict_keys(['EXPERIMENT', 'SUBMISSION', 'Organization', 'STUDY', 'SAMPLE', 'Pool', 'RUN_SET'])

In [34]:
for key in fetch_result['EXPERIMENT_PACKAGE_SET']['EXPERIMENT_PACKAGE'].keys():
    print('key values: ', key)
    print(fetch_result['EXPERIMENT_PACKAGE_SET']['EXPERIMENT_PACKAGE'][key].keys())

key values:  EXPERIMENT
odict_keys(['@accession', '@alias', 'IDENTIFIERS', 'TITLE', 'STUDY_REF', 'DESIGN', 'PLATFORM'])
key values:  SUBMISSION
odict_keys(['@lab_name', '@center_name', '@accession', '@alias', 'IDENTIFIERS'])
key values:  Organization
odict_keys(['@type', 'Name', 'Address', 'Contact'])
key values:  STUDY
odict_keys(['@center_name', '@alias', '@accession', 'IDENTIFIERS', 'DESCRIPTOR'])
key values:  SAMPLE
odict_keys(['@alias', '@accession', 'IDENTIFIERS', 'TITLE', 'SAMPLE_NAME', 'SAMPLE_LINKS', 'SAMPLE_ATTRIBUTES'])
key values:  Pool
odict_keys(['Member'])
key values:  RUN_SET
odict_keys(['RUN'])


In [38]:
print(fetch_result['EXPERIMENT_PACKAGE_SET']['EXPERIMENT_PACKAGE']['EXPERIMENT']['@accession'])
print(fetch_result['EXPERIMENT_PACKAGE_SET']['EXPERIMENT_PACKAGE']['SUBMISSION']['@accession'])
print(fetch_result['EXPERIMENT_PACKAGE_SET']['EXPERIMENT_PACKAGE']['STUDY']['@accession'])
print(fetch_result['EXPERIMENT_PACKAGE_SET']['EXPERIMENT_PACKAGE']['SAMPLE']['@accession'])

SRX3422726
SRA633978
SRP125650
SRS2714377


In [40]:
st = 'SRS2714377'

In [42]:
st[3::2]

'2137'

In [43]:
def chunks(l, n):
    """Yield successive n-sized chunks from l."""
    for i in range(0, len(l), n):
        yield l[i:i + n]

In [48]:
x = chunks(st[3:], 2)
print([i for i in x])

['27', '14', '37', '7']


In [30]:
fetch_result['EXPERIMENT_PACKAGE_SET']['EXPERIMENT_PACKAGE']#['EXPERIMENT']

OrderedDict([('EXPERIMENT',
              OrderedDict([('@accession', 'SRX3422726'),
                           ('@alias', '9'),
                           ('IDENTIFIERS',
                            OrderedDict([('PRIMARY_ID', 'SRX3422726'),
                                         ('SUBMITTER_ID',
                                          OrderedDict([('@namespace',
                                                        'SUB3255603'),
                                                       ('#text', '9')]))])),
                           ('TITLE',
                            'Plant sample from Arabidopsis thaliana WTA-3'),
                           ('STUDY_REF',
                            OrderedDict([('@accession', 'SRP125650'),
                                         ('IDENTIFIERS',
                                          OrderedDict([('PRIMARY_ID',
                                                        'SRP125650'),
                                                       ('EX

In [28]:
fetch_result['EXPERIMENT_PACKAGE_SET']['EXPERIMENT_PACKAGE']\
        ['EXPERIMENT']['@accession']

'SRX3422726'